In [50]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [51]:
# make dictionary of sensitivity values per environment
sensitivity = {'LANL':{'Mars':{'braga':0.000245701982436336, 'metals':0.0000919578459982072}},
               'ChemLIBS':{'Mars':{'braga':0.00028197645372017, 'metals':0.000117639483634021},
               'Earth':{'braga':0.000444867993101603, 'metals':0.0000931716553399781},
               'Vacuum':{'braga':0.000417553099169204, 'metals':0.000231951753003813}}}

In [52]:
# make function that returns values per environment
def get_results(instrument, atmosphere):
    
    braga_sens = sensitivity[instrument][atmosphere]['braga']
    metals_sens = sensitivity[instrument][atmosphere]['metals']
    
    for n_range in ['0-750', '250-1000']:
        print('Calculating', instrument, atmosphere, n_range, "values")
        folder = "C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\"+instrument+" calculations\\models\\"+atmosphere+"\\"+n_range+"\\"
        coefflist = os.listdir(folder)
        coeffs = []

        # read models
        for file in tqdm(coefflist):
            if "coeff" in file:       
                path = folder + file
                data = pd.read_csv(path, skiprows = [0])
                coeffs.append(data)

        # convert to dataframe
        coeffs = pd.concat(coeffs).T
        
        # adapt ot different element naming b/w datasets
        if instrument == 'ChemLIBS':
            coeffs.columns = coeffs.iloc[0].map(lambda x: x.split(':')[1])
        else: coeffs.columns = coeffs.iloc[0].map(lambda x: x.split()[0])
            
        coeffs = coeffs.drop(coeffs.index[0])

        # calculate regression vectors
        vector_list = coeffs.pow(2).sum().pow(.5)  #square root of sum of squares

        # populate lists
        elem_list = coeffs.columns
        instr_list = [instrument] * len(vector_list)
        atm_list = [atmosphere] * len(vector_list)
        range_list = [n_range] * len(vector_list)

        temp = pd.DataFrame({'element' : elem_list,
                             'instrument' : instr_list,
                             'atmosphere' : atm_list,
                             'num_range' : range_list,
                             'vector' : vector_list
        }).reset_index(drop = True)

        # calculate values
        types = ['LOB', 'LOD', 'LOQ']
        factors = [1.645, 3.3, 10]
        
        for i in range(len(types)):
            temp[types[i]+"_Braga"] = factors[i] * braga_sens * temp['vector']
            temp[types[i]+"_metals"] = factors[i] * metals_sens * temp['vector']
        
        df = temp if n_range == '0-750' else pd.concat([df, temp])

    # return table
    return df

In [53]:
envs = [['ChemLIBS', 'Mars'], ['ChemLIBS', 'Earth'], ['ChemLIBS', 'Vacuum'], ['LANL', 'Mars']]

for env in envs:
    results = get_results(env[0], env[1])
    full_results = results if env == envs[0] else pd.concat[full_results, results].reset_index(drop=True)
    
display(full_results)

Calculating ChemLIBS Mars 0-750 values


  0%|          | 0/28 [00:00<?, ?it/s]

Calculating ChemLIBS Mars 250-1000 values


  0%|          | 0/28 [00:00<?, ?it/s]

Calculating ChemLIBS Earth 0-750 values


...

Calculating LANL Mars 0-750 values


  0%|          | 0/37 [00:00<?, ?it/s]

Calculating LANL Mars 250-1000 values


  0%|          | 0/37 [00:00<?, ?it/s]

,element,instrument,atmosphere,num_range,vector,LOB_Braga,LOB_metals,LOD_Braga,LOD_metals,LOQ_Braga,LOQ_metals
0,Li,ChemLIBS,Mars,0-750,8164.69898,3.787206,1.580008,7.597434,3.16962,23.022529,9.60491
1,MnO,ChemLIBS,Mars,0-750,260.473316,0.120821,0.050406,0.242376,0.101118,0.734473,0.306419
2,Na2O,ChemLIBS,Mars,0-750,826.045923,0.383162,0.159854,0.768654,0.32068,2.329255,0.971756
3,Ni,ChemLIBS,Mars,0-750,15529.825718,7.203529,3.005285,14.450849,6.028838,43.790452,18.269207
4,Pb,ChemLIBS,Mars,0-750,880.090905,0.408231,0.170313,0.818944,0.34166,2.481649,1.035334
...,...,...,...,...,...,...,...,...,...,...,...
67,Pb,LANL,Mars,250-1000,520.481655,0.210368,0.078734,0.422015,0.157946,1.278834,0.478624
68,Rb,LANL,Mars,250-1000,7499.395355,3.031104,1.134438,6.080634,2.275773,18.426163,6.896282
69,SiO2,LANL,Mars,250-1000,8542.342845,3.452642,1.292206,6.926273,2.592267,20.988706,7.855354
70,Sr,LANL,Mars,250-1000,111411.378542,45.030224,16.853272,90.334189,33.808996,273.739966,102.451504


In [54]:
# export
#epath = "C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\LOD_calcs.csv"
#full_results.to_csv(epath, index=False)